In [1]:
import requests
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
file = open('database.txt')

lines = file.readlines()
url = lines[0].rstrip()
driver = lines[1].rstrip()
user = lines[2].rstrip()
password = lines[3].rstrip()

In [3]:
spark = SparkSession.builder.appName('CreditCardSystem').getOrCreate()

In [4]:
api_url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'

In [5]:
response = requests.get(api_url)
data = response.json()

print('Status Code is: ' + str(response.status_code))

Status Code is: 200


In [6]:
pd_df = pd.json_normalize(data)

In [7]:
df = spark.createDataFrame(pd_df)

In [11]:
#Work around, since for some god damn reason it doesnt work like intended and crashes pyspark when I used the data from the API
test_df = spark.read.json('test.json')

In [14]:
#It breaks when I name the database table with the required name, so I used a placeholder name of 'loan' and renamed it in HeidiDB
test_df.write.format('jdbc').options(
      url=url,
      driver=driver,
      dbtable='loan',
      user=user,
      password=password).mode('overwrite').save()